# Ejemplo Completo: Clustering No Supervisado
## Segmentación de Clientes con K-Means

### Objetivo
Agrupar clientes según sus patrones de compra sin etiquetas previas, usando aprendizaje no supervisado.

### Conceptos que aprenderás:
- Crear datos sintéticos para clustering
- Implementar K-Means clustering
- Determinar el número óptimo de clusters (método del codo)
- Evaluar clustering con Silhouette Score
- Interpretar y visualizar clusters
- Aplicar clustering a nuevos datos

---

## 1. Importar Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.cm import get_cmap

# Configuración para gráficos
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

---

## 2. Crear Datos Sintéticos de Clientes

Vamos a simular 300 clientes con diferentes patrones de compra para demostrar clustering.

In [ ]:
# Fijar semilla para reproducibilidad
np.random.seed(42)

print("=" * 70)
print("GENERACIÓN DE DATOS SINTÉTICOS")
print("=" * 70)

# Grupo 1: Clientes VIP (compran mucho, gastan mucho)
print("\nGenerando Grupo 1: Clientes VIP...")
n_vip = 100
grupo1_compras = np.random.normal(50, 10, n_vip)      # ~50 compras/mes
grupo1_gasto = np.random.normal(5000, 1000, n_vip)    # ~$5000/mes

# Grupo 2: Clientes Regulares (compras y gastos moderados)
print("Generando Grupo 2: Clientes Regulares...")
n_regular = 100
grupo2_compras = np.random.normal(20, 5, n_regular)   # ~20 compras/mes
grupo2_gasto = np.random.normal(2000, 500, n_regular) # ~$2000/mes

# Grupo 3: Clientes Ocasionales (pocas compras, poco gasto)
print("Generando Grupo 3: Clientes Ocasionales...")
n_ocasional = 100
grupo3_compras = np.random.normal(5, 2, n_ocasional)  # ~5 compras/mes
grupo3_gasto = np.random.normal(500, 200, n_ocasional) # ~$500/mes

# Combinar todos los datos
compras_mensuales = np.concatenate([grupo1_compras, grupo2_compras, grupo3_compras])
gasto_total = np.concatenate([grupo1_gasto, grupo2_gasto, grupo3_gasto])

# Crear DataFrame
df_clientes = pd.DataFrame({
    'compras_mensuales': compras_mensuales,
    'gasto_total': gasto_total
})

print(f"\n✓ Dataset creado: {len(df_clientes)} clientes")
print(f"  • Clientes VIP: {n_vip}")
print(f"  • Clientes Regulares: {n_regular}")
print(f"  • Clientes Ocasionales: {n_ocasional}")

---

## 3. Explorar los Datos

In [ ]:
print("\n" + "=" * 70)
print("DATASET DE CLIENTES")
print("=" * 70)
print("\nPrimeras 15 filas:")
print(df_clientes.head(15))

print("\n" + "=" * 70)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 70)
print(df_clientes.describe())

print("\n" + "=" * 70)
print("INFORMACIÓN DEL DATASET")
print("=" * 70)
print(df_clientes.info())

In [ ]:
# Visualización inicial de los datos
plt.figure(figsize=(10, 6))
plt.scatter(df_clientes['compras_mensuales'], df_clientes['gasto_total'], 
            alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
plt.xlabel('Compras Mensuales', fontsize=12)
plt.ylabel('Gasto Total ($)', fontsize=12)
plt.title('Distribución de Clientes (Sin Clustering)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Distribuciones individuales
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de compras
axes[0].hist(df_clientes['compras_mensuales'], bins=30, color='skyblue', 
             edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Compras Mensuales')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Compras Mensuales', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Distribución de gastos
axes[1].hist(df_clientes['gasto_total'], bins=30, color='lightcoral', 
             edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Gasto Total ($)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Gasto Total', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 4. Preparar los Datos

En clustering, el escalado es CRÍTICO porque K-Means es sensible a la escala de las variables.

In [ ]:
# Extraer características
X = df_clientes.values

print("\n" + "=" * 70)
print("PREPARACIÓN DE DATOS")
print("=" * 70)
print(f"\nForma de X: {X.shape}")
print(f"Características: {df_clientes.columns.tolist()}")

# Escalar características (CRÍTICO para K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\nEstadísticas ANTES del escalado:")
print(f"  Media: {X.mean(axis=0)}")
print(f"  Desviación estándar: {X.std(axis=0)}")
print(f"  Mínimo: {X.min(axis=0)}")
print(f"  Máximo: {X.max(axis=0)}")

print("\nEstadísticas DESPUÉS del escalado:")
print(f"  Media: {X_scaled.mean(axis=0)}")
print(f"  Desviación estándar: {X_scaled.std(axis=0)}")
print(f"  Mínimo: {X_scaled.min(axis=0)}")
print(f"  Máximo: {X_scaled.max(axis=0)}")

---

## 5. Método del Codo para Determinar K Óptimo

Probaremos diferentes números de clusters para encontrar el valor óptimo de K.

#### Métricas para determinar el número óptimo de clusters

- **Inercia**: Es la suma de las distancias al cuadrado entre cada punto y el centro de su cluster. Mide la compactación interna de los clusters. Cuanto menor sea la inercia, más agrupados están los puntos.  
  Se utiliza en el **método del codo** para detectar el punto donde agregar más clusters deja de mejorar significativamente la agrupación.

- **Silhouette Score**: Mide qué tan bien está agrupado cada punto, comparando la distancia con su propio cluster y con el más cercano.  
  Su valor varía entre -1 y 1:
  - Cercano a **1**: buena separación entre clusters.
  - Cercano a **0**: los clusters se solapan.
  - Cercano a **-1**: mala asignación (el punto está más cerca de otro cluster que del suyo).

> En conjunto, estas métricas ayudan a elegir un valor de \( K \) que logre buena compactación sin sobreajuste, y buena separación entre grupos.

In [ ]:
print("\n" + "=" * 70)
print("MÉTODO DEL CODO - DETERMINACIÓN DE K ÓPTIMO")
print("=" * 70)

# Probar diferentes números de clusters
rango_k = range(2, 11)
inercias = []
silhouette_scores = []

print("\nProbando diferentes valores de K...")
for k in rango_k:
    # Crear y entrenar modelo
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    
    # Guardar inercia (suma de distancias al cuadrado)
    inercias.append(kmeans.inertia_)
    
    # Calcular silhouette score
    silhouette = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(silhouette)
    
    print(f"  k={k}: Inercia={kmeans.inertia_:.2f}, Silhouette={silhouette:.4f}")

print("\n✓ Análisis completado")

In [ ]:
# Visualizar método del codo y silhouette scores
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico del codo (Inercia)
axes[0].plot(rango_k, inercias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Número de Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inercia (Within-Cluster Sum of Squares)', fontsize=12)
axes[0].set_title('Método del Codo', fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(rango_k)

# Gráfico de Silhouette Score
axes[1].plot(rango_k, silhouette_scores, 'ro-', linewidth=2, markersize=8)
axes[1].set_xlabel('Número de Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score por K', fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(rango_k)
axes[1].axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Umbral aceptable')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n" + "-" * 70)
print("INTERPRETACIÓN:")
print("-" * 70)
print("• Método del Codo: Buscar el 'codo' donde la inercia deja de disminuir abruptamente")
print("• Silhouette Score: Valores cercanos a 1 indican clusters bien separados")
print("• Recomendación: K=3 parece ser el óptimo en este caso")

---

## 6. Aplicar K-Means con K=3

#### Aplicación de K-Means con número óptimo de clusters

Se entrena el modelo K-Means con el valor óptimo de \( K \) (determinado previamente con el método del codo y el silhouette score). El modelo agrupa los datos en clusters y asigna una etiqueta a cada muestra.

Pasos realizados:

1. **Inicialización del modelo**:
   - `n_clusters = 3`: número de grupos.
   - `random_state = 42`: asegura reproducibilidad.
   - `n_init = 10`: número de inicializaciones para elegir la mejor.

2. **Entrenamiento y predicción**:
   - `fit_predict(X_scaled)`: ajusta el modelo y asigna cada muestra a un cluster.

3. **Resultados**:
   - Se agrega la columna `'cluster'` al `DataFrame` original.
   - Se imprime el número de iteraciones realizadas (`n_iter_`) y la inercia final (`inertia_`), que indica la compactación de los clusters.

Este proceso permite segmentar los datos y analizar patrones dentro de cada grupo.

In [ ]:
# Aplicar K-Means con el número óptimo de clusters
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

print("\n" + "=" * 70)
print("APLICANDO K-MEANS CLUSTERING")
print("=" * 70)
print(f"\nNúmero de clusters: {n_clusters}")
print("Entrenando modelo...")

# Entrenar y predecir clusters
clusters = kmeans.fit_predict(X_scaled)

# Agregar clusters al DataFrame original
df_clientes['cluster'] = clusters

print("✓ Clustering completado")
print(f"\nNúmero de iteraciones: {kmeans.n_iter_}")
print(f"Inercia final: {kmeans.inertia_:.2f}")

---

## 7. Evaluar el Clustering

In [ ]:
# Calcular Silhouette Score
silhouette = silhouette_score(X_scaled, clusters)

print("\n" + "=" * 70)
print("EVALUACIÓN DEL CLUSTERING")
print("=" * 70)
print(f"\nCoeficiente de Silhouette: {silhouette:.4f}")

print("\n" + "-" * 70)
print("INTERPRETACIÓN DEL SILHOUETTE SCORE:")
print("-" * 70)
if silhouette > 0.7:
    print("• Excelente: Clusters muy bien definidos y separados")
elif silhouette > 0.5:
    print("• Bueno: Clusters razonablemente separados")
elif silhouette > 0.25:
    print("• Aceptable: Algunos puntos podrían estar en clusters incorrectos")
else:
    print("• Pobre: Clustering no es adecuado para estos datos")

print(f"\n→ En este caso: {silhouette:.4f} indica {'un buen clustering' if silhouette > 0.5 else 'clustering aceptable'}")

In [ ]:
# Silhouette plot detallado
fig, ax = plt.subplots(figsize=(10, 6))

# Calcular silhouette por muestra
silhouette_vals = silhouette_samples(X_scaled, clusters)

y_lower = 10
for i in range(n_clusters):
    # Valores de silhouette para cluster i
    cluster_silhouette_vals = silhouette_vals[clusters == i]
    cluster_silhouette_vals.sort()
    
    size_cluster_i = cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = get_cmap('Set2')(i / n_clusters)
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_vals,
                      facecolor=color, edgecolor=color, alpha=0.7)
    
    # Etiqueta con el número de cluster
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

ax.set_title('Silhouette Plot para Cada Cluster', fontweight='bold')
ax.set_xlabel('Coeficiente de Silhouette')
ax.set_ylabel('Cluster')
ax.axvline(x=silhouette, color="red", linestyle="--", label=f'Score promedio: {silhouette:.3f}')
ax.set_yticks([])
ax.legend()
plt.tight_layout()
plt.show()

---

## 8. Análisis de los Clusters

In [ ]:
print("\n" + "=" * 70)
print("CARACTERÍSTICAS DE CADA CLUSTER")
print("=" * 70)

for i in range(n_clusters):
    cluster_data = df_clientes[df_clientes['cluster'] == i]
    print(f"\n{'='*70}")
    print(f"CLUSTER {i}")
    print('='*70)
    print(f"Número de clientes: {len(cluster_data)} ({len(cluster_data)/len(df_clientes)*100:.1f}%)")
    print(f"\nEstadísticas:")
    print(f"  Compras mensuales:")
    print(f"    • Promedio: {cluster_data['compras_mensuales'].mean():.2f}")
    print(f"    • Mediana: {cluster_data['compras_mensuales'].median():.2f}")
    print(f"    • Desv. Std: {cluster_data['compras_mensuales'].std():.2f}")
    print(f"  Gasto total:")
    print(f"    • Promedio: ${cluster_data['gasto_total'].mean():,.2f}")
    print(f"    • Mediana: ${cluster_data['gasto_total'].median():,.2f}")
    print(f"    • Desv. Std: ${cluster_data['gasto_total'].std():,.2f}")

In [ ]:
# Tabla resumen comparativa
print("\n" + "=" * 70)
print("TABLA COMPARATIVA DE CLUSTERS")
print("=" * 70)

resumen = df_clientes.groupby('cluster').agg({
    'compras_mensuales': ['count', 'mean', 'median', 'std'],
    'gasto_total': ['mean', 'median', 'std']
}).round(2)

print("\n", resumen)

---

## 9. Visualizar los Clusters

In [ ]:
# Obtener centroides en escala original
centroides_scaled = kmeans.cluster_centers_
centroides_original = scaler.inverse_transform(centroides_scaled)

print("\n" + "=" * 70)
print("CENTROIDES DE LOS CLUSTERS")
print("=" * 70)
for i, centroide in enumerate(centroides_original):
    print(f"\nCluster {i}:")
    print(f"  • Compras mensuales: {centroide[0]:.2f}")
    print(f"  • Gasto total: ${centroide[1]:,.2f}")

In [ ]:
# Visualización de clusters
plt.figure(figsize=(12, 7))

# Colores para cada cluster
colores = ['#FF6B6B', '#4ECDC4', '#45B7D1']
nombres_clusters = ['Cluster 0', 'Cluster 1', 'Cluster 2']

# Plotear cada cluster
for i in range(n_clusters):
    cluster_data = df_clientes[df_clientes['cluster'] == i]
    plt.scatter(cluster_data['compras_mensuales'], cluster_data['gasto_total'],
               c=colores[i], label=nombres_clusters[i], alpha=0.6, s=50,
               edgecolors='black', linewidth=0.5)

# Plotear centroides
plt.scatter(centroides_original[:, 0], centroides_original[:, 1],
           c='red', marker='X', s=300, edgecolors='black', linewidth=2,
           label='Centroides', zorder=5)

# Añadir círculos alrededor de los centroides
for centroide in centroides_original:
    circle = plt.Circle((centroide[0], centroide[1]), 5, color='red', 
                        fill=False, linestyle='--', linewidth=2, alpha=0.5)
    plt.gca().add_patch(circle)

plt.xlabel('Compras Mensuales', fontsize=12)
plt.ylabel('Gasto Total ($)', fontsize=12)
plt.title('Segmentación de Clientes con K-Means (K=3)', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Gráficos de distribución por cluster
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Compras mensuales por cluster
for i in range(n_clusters):
    cluster_data = df_clientes[df_clientes['cluster'] == i]
    axes[0].hist(cluster_data['compras_mensuales'], bins=15, alpha=0.6,
                label=f'Cluster {i}', color=colores[i], edgecolor='black')
axes[0].set_xlabel('Compras Mensuales')
axes[0].set_ylabel('Frecuencia')
axes[0].set_title('Distribución de Compras por Cluster', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gasto total por cluster
for i in range(n_clusters):
    cluster_data = df_clientes[df_clientes['cluster'] == i]
    axes[1].hist(cluster_data['gasto_total'], bins=15, alpha=0.6,
                label=f'Cluster {i}', color=colores[i], edgecolor='black')
axes[1].set_xlabel('Gasto Total ($)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Gastos por Cluster', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots para comparar clusters
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot de compras
df_clientes.boxplot(column='compras_mensuales', by='cluster', ax=axes[0])
axes[0].set_title('Compras Mensuales por Cluster')
axes[0].set_xlabel('Cluster')
axes[0].set_ylabel('Compras Mensuales')
plt.sca(axes[0])
plt.xticks([1, 2, 3], ['Cluster 0', 'Cluster 1', 'Cluster 2'])

# Boxplot de gastos
df_clientes.boxplot(column='gasto_total', by='cluster', ax=axes[1])
axes[1].set_title('Gasto Total por Cluster')
axes[1].set_xlabel('Cluster')
axes[1].set_ylabel('Gasto Total ($)')
plt.sca(axes[1])
plt.xticks([1, 2, 3], ['Cluster 0', 'Cluster 1', 'Cluster 2'])

plt.suptitle('')  # Remover título automático
plt.tight_layout()
plt.show()

---

## 10. Interpretación de Negocios

In [ ]:
print("\n" + "=" * 70)
print("INTERPRETACIÓN DE SEGMENTOS DE CLIENTES")
print("=" * 70)

# Determinar qué cluster corresponde a qué tipo de cliente
clusters_ordenados = []
for i in range(n_clusters):
    cluster_data = df_clientes[df_clientes['cluster'] == i]
    avg_compras = cluster_data['compras_mensuales'].mean()
    avg_gasto = cluster_data['gasto_total'].mean()
    clusters_ordenados.append((i, avg_compras, avg_gasto))

# Ordenar por gasto (de mayor a menor)
clusters_ordenados.sort(key=lambda x: x[2], reverse=True)

# Asignar nombres
interpretaciones = {
    clusters_ordenados[0][0]: "VIP - Alto Valor",
    clusters_ordenados[1][0]: "Regulares - Valor Medio",
    clusters_ordenados[2][0]: "Ocasionales - Bajo Valor"
}

for cluster_id, nombre in interpretaciones.items():
    cluster_data = df_clientes[df_clientes['cluster'] == cluster_id]
    print(f"\n{'='*70}")
    print(f"CLUSTER {cluster_id}: {nombre}")
    print('='*70)
    print(f"Tamaño: {len(cluster_data)} clientes ({len(cluster_data)/len(df_clientes)*100:.1f}%)")
    print(f"Compras mensuales promedio: {cluster_data['compras_mensuales'].mean():.2f}")
    print(f"Gasto total promedio: ${cluster_data['gasto_total'].mean():,.2f}")
    print(f"Valor total del segmento: ${(cluster_data['gasto_total'].sum()):,.2f}")
    
    # Recomendaciones de negocio
    print(f"\nRecomendaciones de Marketing:")
    if "VIP" in nombre:
        print("  • Programa de fidelización premium")
        print("  • Atención personalizada y exclusiva")
        print("  • Acceso anticipado a nuevos productos")
        print("  • Descuentos en compras por volumen")
    elif "Regulares" in nombre:
        print("  • Programa de puntos y recompensas")
        print("  • Ofertas especiales mensuales")
        print("  • Cross-selling de productos complementarios")
        print("  • Incentivos para aumentar frecuencia de compra")
    else:
        print("  • Campañas de reactivación")
        print("  • Descuentos para primera compra recurrente")
        print("  • Email marketing con ofertas atractivas")
        print("  • Identificar barreras de compra y solucionarlas")

---

## 11. Predicción de Cluster para Nuevos Clientes

In [ ]:
print("\n" + "=" * 70)
print("PREDICCIÓN DE CLUSTER PARA NUEVOS CLIENTES")
print("=" * 70)

# Crear nuevos clientes para clasificar
nuevos_clientes = np.array([
    [30, 3500],   # Cliente 1: 30 compras, $3500
    [8, 800],     # Cliente 2: 8 compras, $800
    [55, 6000],   # Cliente 3: 55 compras, $6000
    [15, 1500]    # Cliente 4: 15 compras, $1500
])

# Escalar los nuevos clientes
nuevos_clientes_scaled = scaler.transform(nuevos_clientes)

# Predecir clusters
clusters_predichos = kmeans.predict(nuevos_clientes_scaled)

# Calcular distancias a cada centroide
distancias = kmeans.transform(nuevos_clientes_scaled)

# Mostrar resultados
for i, (cliente, cluster_pred, dists) in enumerate(zip(nuevos_clientes, clusters_predichos, distancias), 1):
    print(f"\n{'='*70}")
    print(f"NUEVO CLIENTE {i}")
    print('='*70)
    print(f"Características:")
    print(f"  • Compras mensuales: {cliente[0]:.0f}")
    print(f"  • Gasto total: ${cliente[1]:,.2f}")
    print(f"\n→ Cluster asignado: {cluster_pred} ({interpretaciones[cluster_pred]})")
    
    print(f"\nDistancias a cada centroide:")
    for j, dist in enumerate(dists):
        print(f"  Cluster {j}: {dist:.4f}")
    
    # Recomendación
    print(f"\nPerfil del cliente: {interpretaciones[cluster_pred]}")

In [ ]:
# Visualizar nuevos clientes en el gráfico
plt.figure(figsize=(12, 7))

# Plotear clusters existentes
for i in range(n_clusters):
    cluster_data = df_clientes[df_clientes['cluster'] == i]
    plt.scatter(cluster_data['compras_mensuales'], cluster_data['gasto_total'],
               c=colores[i], label=f'Cluster {i} - {interpretaciones[i]}', 
               alpha=0.4, s=50, edgecolors='black', linewidth=0.5)

# Plotear centroides
plt.scatter(centroides_original[:, 0], centroides_original[:, 1],
           c='red', marker='X', s=300, edgecolors='black', linewidth=2,
           label='Centroides', zorder=5)

# Plotear nuevos clientes
plt.scatter(nuevos_clientes[:, 0], nuevos_clientes[:, 1],
           c='yellow', marker='*', s=500, edgecolors='black', linewidth=2,
           label='Nuevos Clientes', zorder=6)

# Anotar nuevos clientes
for i, (x, y) in enumerate(nuevos_clientes, 1):
    plt.annotate(f'Nuevo {i}', (x, y), xytext=(5, 5), 
                textcoords='offset points', fontsize=10, fontweight='bold')

plt.xlabel('Compras Mensuales', fontsize=12)
plt.ylabel('Gasto Total ($)', fontsize=12)
plt.title('Clientes Existentes vs Nuevos Clientes', fontsize=14, fontweight='bold')
plt.legend(fontsize=9, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## 12. Análisis de Estabilidad del Clustering

In [ ]:
print("\n" + "=" * 70)
print("ANÁLISIS DE ESTABILIDAD - MÚLTIPLES EJECUCIONES")
print("=" * 70)

# Ejecutar K-Means múltiples veces con diferentes inicializaciones
n_ejecuciones = 10
inercias_multiples = []
silhouette_multiples = []

print(f"\nEjecutando K-Means {n_ejecuciones} veces...")

for i in range(n_ejecuciones):
    kmeans_temp = KMeans(n_clusters=3, random_state=i, n_init=10)
    labels_temp = kmeans_temp.fit_predict(X_scaled)
    
    inercias_multiples.append(kmeans_temp.inertia_)
    silhouette_multiples.append(silhouette_score(X_scaled, labels_temp))

print("\nResultados de múltiples ejecuciones:")
print(f"  Inercia:")
print(f"    • Media: {np.mean(inercias_multiples):.2f}")
print(f"    • Desv. Std: {np.std(inercias_multiples):.2f}")
print(f"    • Rango: [{np.min(inercias_multiples):.2f}, {np.max(inercias_multiples):.2f}]")

print(f"\n  Silhouette Score:")
print(f"    • Media: {np.mean(silhouette_multiples):.4f}")
print(f"    • Desv. Std: {np.std(silhouette_multiples):.4f}")
print(f"    • Rango: [{np.min(silhouette_multiples):.4f}, {np.max(silhouette_multiples):.4f}]")

if np.std(silhouette_multiples) < 0.01:
    print("\n✓ Clustering estable: Resultados consistentes entre ejecuciones")
else:
    print("\n⚠ Clustering variable: Considerar ajustar parámetros o aumentar n_init")

In [ ]:
# Visualizar variabilidad
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot de inercias
axes[0].boxplot([inercias_multiples], labels=['Inercia'])
axes[0].set_ylabel('Valor')
axes[0].set_title('Variabilidad de Inercia', fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Boxplot de silhouette scores
axes[1].boxplot([silhouette_multiples], labels=['Silhouette'])
axes[1].set_ylabel('Valor')
axes[1].set_title('Variabilidad de Silhouette Score', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 13. Comparación con Otros Algoritmos de Clustering

In [ ]:
from sklearn.cluster import DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture

print("\n" + "=" * 70)
print("COMPARACIÓN CON OTROS ALGORITMOS DE CLUSTERING")
print("=" * 70)

# K-Means (ya lo tenemos)
kmeans_labels = kmeans.labels_
kmeans_silhouette = silhouette_score(X_scaled, kmeans_labels)

# DBSCAN
print("\nEjecutando DBSCAN...")
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X_scaled)
if len(set(dbscan_labels)) > 1:
    dbscan_silhouette = silhouette_score(X_scaled, dbscan_labels)
else:
    dbscan_silhouette = -1

# Hierarchical Clustering
print("Ejecutando Hierarchical Clustering...")
hierarchical = AgglomerativeClustering(n_clusters=3)
hierarchical_labels = hierarchical.fit_predict(X_scaled)
hierarchical_silhouette = silhouette_score(X_scaled, hierarchical_labels)

# Gaussian Mixture
print("Ejecutando Gaussian Mixture Model...")
gmm = GaussianMixture(n_components=3, random_state=42)
gmm_labels = gmm.fit_predict(X_scaled)
gmm_silhouette = silhouette_score(X_scaled, gmm_labels)

# Resultados
print("\n" + "=" * 70)
print("RESULTADOS COMPARATIVOS")
print("=" * 70)

resultados_clustering = {
    'K-Means': {
        'silhouette': kmeans_silhouette,
        'n_clusters': len(set(kmeans_labels)),
        'labels': kmeans_labels
    },
    'DBSCAN': {
        'silhouette': dbscan_silhouette,
        'n_clusters': len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0),
        'labels': dbscan_labels
    },
    'Hierarchical': {
        'silhouette': hierarchical_silhouette,
        'n_clusters': len(set(hierarchical_labels)),
        'labels': hierarchical_labels
    },
    'Gaussian Mixture': {
        'silhouette': gmm_silhouette,
        'n_clusters': len(set(gmm_labels)),
        'labels': gmm_labels
    }
}

print(f"\n{'Algoritmo':<20} {'N° Clusters':<15} {'Silhouette Score':<20}")
print("-" * 55)
for nombre, resultado in resultados_clustering.items():
    print(f"{nombre:<20} {resultado['n_clusters']:<15} {resultado['silhouette']:<20.4f}")

mejor_algoritmo = max(resultados_clustering.items(), key=lambda x: x[1]['silhouette'])
print(f"\n→ Mejor algoritmo: {mejor_algoritmo[0]} (Silhouette: {mejor_algoritmo[1]['silhouette']:.4f})")


In [ ]:
# Visualizar comparación de algoritmos
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

algoritmos = ['K-Means', 'DBSCAN', 'Hierarchical', 'Gaussian Mixture']
for idx, (nombre, ax) in enumerate(zip(algoritmos, axes.flatten())):
    labels = resultados_clustering[nombre]['labels']
    
    # Manejar outliers de DBSCAN (-1)
    unique_labels = set(labels)
    colors_map = plt.cm.Set2(np.linspace(0, 1, len(unique_labels)))
    
    for k, col in zip(unique_labels, colors_map):
        if k == -1:
            # Outliers en negro
            col = [0, 0, 0, 1]
            label = 'Outliers'
        else:
            label = f'Cluster {k}'
        
        class_member_mask = (labels == k)
        xy = X[class_member_mask]
        ax.scatter(xy[:, 0], xy[:, 1], c=[col], label=label, 
                  alpha=0.6, s=50, edgecolors='black', linewidth=0.5)
    
    ax.set_xlabel('Compras Mensuales')
    ax.set_ylabel('Gasto Total ($)')
    ax.set_title(f'{nombre}\nSilhouette: {resultados_clustering[nombre]["silhouette"]:.4f}', 
                fontweight='bold')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

## 14. Exportar Resultados

In [ ]:
print("\n" + "=" * 70)
print("EXPORTAR RESULTADOS")
print("=" * 70)

# Añadir interpretación al DataFrame
df_clientes['segmento'] = df_clientes['cluster'].map(interpretaciones)

# Mostrar resumen final
print("\nDataFrame con segmentación:")
print(df_clientes.head(15))

# Guardar a CSV
nombre_archivo = 'clientes_segmentados.csv'
df_clientes.to_csv(nombre_archivo, index=False)
print(f"\n✓ Datos guardados en '{nombre_archivo}'")

# Resumen ejecutivo
print("\n" + "=" * 70)
print("RESUMEN EJECUTIVO")
print("=" * 70)
print(f"\nTotal de clientes analizados: {len(df_clientes)}")
print(f"Número de segmentos identificados: {n_clusters}")
print(f"Calidad del clustering (Silhouette): {silhouette:.4f}")

print("\nDistribución de clientes por segmento:")
for cluster_id, nombre in interpretaciones.items():
    count = len(df_clientes[df_clientes['cluster'] == cluster_id])
    porcentaje = count / len(df_clientes) * 100
    valor_total = df_clientes[df_clientes['cluster'] == cluster_id]['gasto_total'].sum()
    print(f"\n{nombre}:")
    print(f"  • Clientes: {count} ({porcentaje:.1f}%)")
    print(f"  • Valor total: ${valor_total:,.2f}")
    print(f"  • Valor promedio por cliente: ${valor_total/count:,.2f}")


---

## 15. Conclusiones

### ✓ Lo que aprendimos:
1. **Crear datos sintéticos** para practicar clustering
2. **Aplicar el método del codo** para determinar K óptimo
3. **Implementar K-Means clustering** con Scikit-Learn
4. **Evaluar clustering** con Silhouette Score
5. **Interpretar clusters** en contexto de negocio
6. **Visualizar resultados** de múltiples formas
7. **Predecir clusters** para nuevos datos
8. **Comparar diferentes algoritmos** de clustering

### 📊 Resultados clave:
- Identificamos 3 segmentos claros de clientes
- El clustering tiene buena calidad (Silhouette > 0.5)
- Cada segmento tiene características distintivas
- Los centroides representan bien cada grupo

### 🎯 Aplicaciones reales del clustering:
- **Marketing**: Segmentación de clientes para campañas personalizadas
- **E-commerce**: Sistemas de recomendación basados en grupos
- **Finanzas**: Detección de patrones de fraude
- **Retail**: Optimización de inventario por zonas
- **Salud**: Agrupación de pacientes para tratamientos
- **Telecomunicaciones**: Identificación de patrones de uso

### 💡 Ventajas del clustering:
- No requiere datos etiquetados (aprendizaje no supervisado)
- Descubre patrones ocultos en los datos
- Escalable a grandes volúmenes de datos
- Útil para exploración y comprensión de datos

### ⚠️ Consideraciones importantes:
- El escalado de datos es CRÍTICO para K-Means
- K-Means es sensible a la inicialización (usar n_init alto)
- Los outliers pueden afectar los resultados
- El número de clusters puede no ser obvio
- Siempre validar con métricas y visualización

### 🎯 Próximos pasos:
- Probar con datasets reales (Customer Segmentation datasets en Kaggle)
- Explorar clustering jerárquico para dendrogramas
- Aplicar DBSCAN para datos con formas irregulares
- Combinar clustering con análisis de componentes principales (PCA)
- Implementar clustering en series temporales

### 💼 Valor de negocio:
El clustering no supervisado permite:
- Entender mejor a los clientes sin etiquetas previas
- Personalizar estrategias de marketing por segmento
- Optimizar recursos focalizándose en segmentos rentables
- Identificar oportunidades de crecimiento
- Mejorar la retención con estrategias segmentadas

---

## 📚 Referencias
- K-Means Algorithm: MacQueen, J. (1967). "Some methods for classification and analysis of multivariate observations"
- Silhouette Score: Rousseeuw, P.J. (1987). "Silhouettes: a graphical aid to the interpretation"
- Documentación Scikit-Learn: https://scikit-learn.org/stable/modules/clustering.html
- Datasets de práctica: Mall Customer Segmentation, Online Retail Dataset (Kaggle)

---

## 🔍 Glosario de Clustering

- **Cluster**: Grupo de puntos similares entre sí
- **Centroide**: Punto central de un cluster (promedio de todos los puntos)
- **Inercia**: Suma de distancias al cuadrado de cada punto a su centroide
- **Silhouette Score**: Métrica que mide qué tan bien está cada punto en su cluster
- **Método del Codo**: Técnica para encontrar el número óptimo de clusters
- **K**: Número de clusters (hiperparámetro en K-Means)
- **Convergencia**: Cuando el algoritmo ya no cambia las asignaciones de clusters
- **Outlier**: Punto que no pertenece claramente a ningún cluster

---

**¡Fin del Notebook de Clustering!**

*Recuerda: El clustering es una herramienta exploratoria poderosa. Siempre valida tus resultados con conocimiento del dominio y múltiples métricas.*